In [ ]:
from urllib.parse import unquote, urlparse


def normalize_url( url: str) -> str:
    """Normalize the SharePoint URL to ensure compatibility"""

    parsed_url = urlparse(url)
    path = unquote(parsed_url.path).strip('/')
    # Construct the Graph API path
    # Assuming the path is something like '/sites/SiteName/Shared Documents/...'
    if path.startswith('sites/'):
        path_parts = path.split('/', 3)
        if len(path_parts) > 3:
            site_path = path_parts[3]  # This should be the path after '/sites/SiteName/'
        else:
            site_path = ''
    else:
        site_path = path

    return site_path


url = "https://datalyactuarial.sharepoint.com/sites/DatalyActuarial/Shared%20Documents/Business%20Development%20and%20Thought%20Leadership/Valuation%20Model%20Development/Run1/models"
normalize_url(url)


In [ ]:
from model_utils import get_model_handler
import pandas as pd
from sharepoint_utils import SharePointClient  # Import directly

# Your token from browser

# Create SharePoint client directly (bypassing get_model_handler)
sp_client = SharePointClient(token=token)

# Test URL
test_url = "https://datalyactuarial.sharepoint.com/sites/DatalyActuarial/Shared%20Documents/Business%20Development%20and%20Thought%20Leadership/Valuation%20Model%20Development/Python%20Models/IP%20Model%20Resources/IP_Model_Source_Data"

def download_assumptions_IP(url):
# download all files in the folder
    files = sp_client.list_files(url)
    assumptions_dict = {}
    for file in files:
        if file.endswith(".xlsx") or file.endswith(".xls"):
            assumption_file = sp_client.download_file(f"{url}/{file}")
            # Get all sheet names
            excel_file = pd.ExcelFile(assumption_file)
            
            # Read each sheet into the dictionary
            for sheet_name in excel_file.sheet_names:
                df = pd.read_excel(assumption_file, sheet_name=sheet_name)
                assumptions_dict[sheet_name] = df
    return assumptions_dict

# Try a simple request
try:
    assumptions_dict = download_assumptions_IP(test_url)
    print("Success!")
    print("\nAssumption tables loaded:")
    for sheet_name, df in assumptions_dict.items():
        print(f"\nSheet: {sheet_name}")
        print(f"Shape: {df.shape}")
except Exception as e:
    print(f"Error: {str(e)}")

In [ ]:
assumptions_dict["Inflation"]

In [59]:
import pandas as pd


def transform_assumptions(assumptions_dict):
    """
    Transform all assumption tables and return them in a dictionary

    Args:
        assumptions_dict: Dictionary of raw assumption DataFrames

    Returns:
        Dictionary of transformed DataFrames
    """
    transformed = {}

    mapping_tables = [
        "Occupation",
        "Waiting_period",
        "Smoker",
        "Benefit_period",
        "Prem_payment_freq",
    ]

    for table in mapping_tables:
        if table in assumptions_dict:
            transformed[table] = assumptions_dict[table].copy()

    # 1. Simple direct assignments (no transformations needed)
    simple_tables = [
        "Mortality",
        "Lapse",
        "TPD",
        "Trauma",
        "Monthly_discount_rates",
        "Commission_rates",
        "Prem_related_expenses",
        "Fixed_expenses",
        "Risk_adj_pc",
        "Variables",
        "Termination_new_claim",
        "Termination_cause_of_sickness",
    ]

    for table in simple_tables:
        if table in assumptions_dict:
            transformed[table] = assumptions_dict[table].copy()

    # Premium rate tables with Y/N to S/N transformation
    premium_tables = [
        "Prem_rate_level",
        "Prem_rate_stepped",
        "Rein_Prem_rate_level",
        "Rein_Prem_rate_stepped",
    ]

    for table in premium_tables:
        if table in assumptions_dict:
            df = assumptions_dict[table].copy()
            # Only transform the 'Smoker status' column
            df["Smoker status"] = df["Smoker status"].map({"Y": "S", "N": "N"})
            transformed[table] = df

    # 2. Death Only Mortality transformations
    df_death_only_mort = assumptions_dict["DeathOnly_mort_age_rates"].copy()
    Death_Only_Mort_Age_Rates = df_death_only_mort.rename(
        columns={
            "Sex": "sex",
            "Age last birthday at last policy anniversary": "Age LB",
            "Non-smoker": "N",
            "Smoker": "S",
        }
    )
    Death_Only_Mort_Age_Rates = Death_Only_Mort_Age_Rates.drop(columns="Aggregate")
    transformed["Death_Only_Mort_Age_Rates"] = pd.melt(
        Death_Only_Mort_Age_Rates,
        id_vars=["Age LB", "sex"],
        var_name="Smoker status",
        value_name="Mortality Age Rates",
    )

    # 3. Death Only Duration Loading
    df_death_only_duration = assumptions_dict["DeathOnly_duration_loading"].copy()
    transformed["Death_Only_Duration_Loading"] = (
        pd.melt(
            df_death_only_duration,
            id_vars=["Policy Duration (Curtate Years)"],
            var_name="sex",
            value_name="Duration Loading",
        )
        .assign(
            **{
                "Policy Duration (Curtate Years)": lambda x: x[
                    "Policy Duration (Curtate Years)"
                ].astype(str)
            }
        )
        .replace({"sex": {"Male": "M", "Female": "F"}})
    )

    # 4. Incidence Age Rates (Female)
    df_incidence_female = assumptions_dict["Incidence_age_rates_females"].copy()
    transformed["Incidence_Age_Rates_Female"] = df_incidence_female.rename(
        columns={
            "Age": "Age LB",
            "Accident": "Accident Age Rates",
            "Sickness": "Sick Age Rates",
        }
    )

    # 5. Incidence Age Rates (Male)
    df_incidence_male = assumptions_dict["Incidence_age_rates_males"].copy()
    male_rates = df_incidence_male.rename(columns={"Age": "Age LB"})
    transformed["Incidence_Age_Rates_Male"] = pd.melt(
        male_rates,
        id_vars=["Sex", "Age LB"],
        var_name="Accident Incidence Type",
        value_name="Accident Age Rates",
    ).replace(
        {
            "Accident Incidence Type": {
                "Accident Combined White Collar": "W",
                "Accident Combined Blue Collar": "B",
                "Sickness": "S",
            }
        }
    )[
        ["Age LB", "Sex", "Accident Incidence Type", "Accident Age Rates"]
    ]

    # 6. Incidence Lifetime Benefit Period
    df_lifetime_benefit = assumptions_dict["Incidence_lifetime_bene_period"].copy()
    transformed["Incidence_Lifetime_Benefit_Period"] = df_lifetime_benefit.rename(
        columns={
            "Accident": "Accident Lifetime Factor",
            "Sickness": "Sick Lifetime Factor",
            "Sex": "sex",
        }
    )

    # 7. Incidence Waiting Period
    df_waiting_period = assumptions_dict["Incidence_waiting_period"].copy()
    occupation_mapping = {
        "Professional/Medical": "P",
        "White Collar": "W",
        "Sedentary": "S",
        "Trades-person": "T",
        "Blue/Heavy Blue Collar": "B",
    }
    waiting_period = pd.melt(
        df_waiting_period,
        id_vars=["Type", "Sex", "Waiting_Period"],
        var_name="Occupation",
        value_name="Waiting Factor",
    )
    waiting_period["Occupation"] = waiting_period["Occupation"].map(occupation_mapping)
    transformed["Incidence_Waiting_Period"] = (
        waiting_period.pivot_table(
            index=["Sex", "Waiting_Period", "Occupation"],
            columns="Type",
            values="Waiting Factor",
        )
        .reset_index()
        .rename(
            columns={"Accident": "Accident Wait Factor", "Sickness": "Sick Wait Factor"}
        )
    )

    # 8. Incidence Smoking Status
    df_smoking_status = assumptions_dict["Incidence_smoking_status"].copy()
    occupation_mapping = {
        "Combined White Collar": ["W", "P"],
        "Combined Blue Collar": ["S", "T", "B"],
    }
    smoking_mapping = {"Smoker": "S", "Non-smoker": "N"}

    smoking_status = pd.melt(
        df_smoking_status,
        id_vars=["Type", "Sex", "Smoking_Status"],
        var_name="Occupation Type",
        value_name="Smoker Factor",
    )
    smoking_status["Smoking_Status"] = smoking_status["Smoking_Status"].map(
        smoking_mapping
    )

    expanded_rows = []
    for _, row in smoking_status.iterrows():
        occupation_codes = occupation_mapping[row["Occupation Type"]]
        for code in occupation_codes:
            new_row = row.copy()
            new_row["Occupation"] = code
            expanded_rows.append(new_row)

    smoking_status_transformed = pd.DataFrame(expanded_rows)
    smoking_status_transformed = smoking_status_transformed.drop(
        columns=["Occupation Type"]
    )
    transformed["Incidence_Smoking_Status"] = (
        smoking_status_transformed.pivot_table(
            index=["Sex", "Smoking_Status", "Occupation"],
            columns="Type",
            values="Smoker Factor",
        )
        .reset_index()
        .rename(
            columns={
                "Accident": "Accident Smoke Factor",
                "Sickness": "Sick Smoke Factor",
            }
        )
    )

    # 9. Incidence Benefit Type
    df_benefit_type = assumptions_dict["Incidence_benefit_type"].copy()
    benefit_type_mapping = {"Agreed Value": "A", "Indemnity": "I"}

    benefit_type = pd.melt(
        df_benefit_type,
        id_vars=["Type", "Sex", "Benefit Type"],
        var_name="Occupation Type",
        value_name="Benefit Type Factor",
    )
    benefit_type["Benefit Type"] = benefit_type["Benefit Type"].map(
        benefit_type_mapping
    )

    expanded_rows = []
    for _, row in benefit_type.iterrows():
        occupation_codes = occupation_mapping[row["Occupation Type"]]
        for code in occupation_codes:
            new_row = row.copy()
            new_row["Occupation"] = code
            expanded_rows.append(new_row)

    benefit_type_transformed = pd.DataFrame(expanded_rows)
    benefit_type_transformed = benefit_type_transformed.drop(
        columns=["Occupation Type"]
    )
    transformed["Incidence_Benefit_Type"] = (
        benefit_type_transformed.pivot_table(
            index=["Sex", "Occupation", "Benefit Type"],
            columns="Type",
            values="Benefit Type Factor",
        )
        .reset_index()
        .rename(
            columns={
                "Accident": "Accident Benefit Type Factor",
                "Sickness": "Sick Benefit Type Factor",
            }
        )
    )

    # 10. Incidence Duration Loading
    df_duration_loading = assumptions_dict["Incidence_duration_loading"].copy()
    transformed["Incidence_Duration_Loading"] = df_duration_loading.assign(
        **{
            "Policy Duration (Curtate Years)": lambda x: x[
                "Policy Duration (Curtate Years)"
            ].astype(str)
        }
    ).rename(
        columns={
            "Accident": "Accident Duration Factor",
            "Sickness": "Sick Duration Factor",
        }
    )

    # 11. Incidence Age Rates Sickness Combined
    df_sickness_female = assumptions_dict["Incidence_age_rates_females"][
        ["Sex", "Age", "Sickness"]
    ].copy()
    df_sickness_male = assumptions_dict["Incidence_age_rates_males"][
        ["Sex", "Age", "Sickness"]
    ].copy()

    df_sickness_female = df_sickness_female.rename(
        columns={"Age": "Age LB", "Sex": "sex", "Sickness": "Sick Age Rates"}
    )
    df_sickness_male = df_sickness_male.rename(
        columns={"Age": "Age LB", "Sex": "sex", "Sickness": "Sick Age Rates"}
    )

    transformed["Incidence_Age_Rates_Sickness_Combined"] = pd.concat(
        [df_sickness_male, df_sickness_female], ignore_index=True
    )

    # 12. Death Only Mortality Floor
    df_mortality_floor = assumptions_dict["DeathOnly_mort_floor"].copy()
    mortality_floor = df_mortality_floor[["Age LB", "225% MS", "225% FS"]].rename(
        columns={"225% MS": "M", "225% FS": "F"}
    )
    transformed["Death_Only_Mortality_Floor"] = pd.melt(
        mortality_floor,
        id_vars=["Age LB"],
        var_name="sex",
        value_name="Mortality Floor",
    )

    # 13. Termination Age Rates
    df_termination_rates = assumptions_dict["Termination_age_rates"].copy()
    termination_rates = df_termination_rates.rename(
        columns={
            "Age last birthday at last policy anniversary at Date of Disablement \ Gender": "Age LB",
            "Male": "M",
            "Female": "F",
        }
    )
    transformed["Termination_Age_Rates"] = pd.melt(
        termination_rates,
        id_vars=["Age LB"],
        var_name="sex",
        value_name="Termination Age Rates",
    )

    # 14. Termination Smoker
    df_termination_smoker = assumptions_dict["Termination_smoker"].copy()
    transformed["Termination_Smoker"] = df_termination_smoker.rename(
        columns={"Smoker Status": "Smoker status", "Rate": "Termination Smoker status"}
    )

    # 15. Termination Benefit Type
    df_termination_benefit = assumptions_dict["Termination_benefit_type"].copy()
    benefit_type_mapping = {"Agreed Value": "A", "Indemnity": "I"}
    transformed["Termination_Benefit_Type"] = df_termination_benefit.rename(
        columns={"Rates": "Termination Benefit Type"}
    ).assign(**{"Benefit Type": lambda x: x["Benefit Type"].map(benefit_type_mapping)})

    # 16. Termination Duration Factor Accident
    df_termination_duration_acc = assumptions_dict[
        "Termination_duration_factor_acc"
    ].copy()
    transformed["Termination_Duration_Factor_Accident"] = (
        df_termination_duration_acc.rename(
            columns={
                "Curtate Policy Year": "Policy Year_10+",
                "Sex": "sex",
                "Rates": "Accident Policy Duration Factor",
            }
        )
        .drop(columns=["Type"])
        .assign(**{"Policy Year_10+": lambda x: x["Policy Year_10+"].astype(str)})
    )

    # 17. Termination Duration Claim Accident
    df_termination_claim_acc = assumptions_dict["Termination_duration_claim_acc"].copy()
    transformed["Termination_Duration_Claim_Acc"] = df_termination_claim_acc.rename(
        columns={
            "Sex": "sex",
            "Waiting_period": "Waiting Period",
            "Rates": "Claim Waiting Occupation Factor",
        }
    ).assign(**{"Claim Duration": lambda x: x["Claim Duration"].astype(int)})

    # 18. Termination Benefit Period
    df_termination_benefit_period = assumptions_dict[
        "Termination_benefit_period"
    ].copy()
    transformed["Termination_Benefit_Period"] = df_termination_benefit_period.rename(
        columns={
            "Duration since Disablement (Years***)": "Claim Duration_6+",
            "Benefit Period": "Benefit Period_65+",
            "Rates": "Benefit Period Factor",
        }
    ).assign(
        **{
            "Claim Duration_6+": lambda x: x["Claim Duration_6+"].astype(str),
            "Benefit Period_65+": lambda x: x["Benefit Period_65+"].astype(str),
        }
    )

    # 19. Termination Duration Factor Sickness
    df_termination_duration_sick = assumptions_dict[
        "Termination_duration_factor_sic"
    ].copy()
    transformed["Termination_Duration_Factor_Sickness"] = (
        df_termination_duration_sick.rename(
            columns={
                "Curtate Policy Year": "Policy Year_10+",
                "Sex": "sex",
                "Rates": "Sickness Policy Duration Factor",
            }
        )
        .drop(columns=["Type"])
        .assign(**{"Policy Year_10+": lambda x: x["Policy Year_10+"].astype(str)})
    )

    # 20. Termination Duration Claim Sickness
    df_termination_claim_sick = assumptions_dict[
        "Termination_duration_claim_sick"
    ].copy()
    transformed["Termination_Duration_Claim_Sick"] = df_termination_claim_sick.rename(
        columns={
            "Sex": "sex",
            "Waiting_period": "Waiting Period",
            "Rates": "Claim Waiting Occupation Factor",
        }
    ).assign(**{"Claim Duration": lambda x: x["Claim Duration"].astype(int)})

    # 21. Inflation
    df_inflation = assumptions_dict["Inflation"].copy()
    # 拆分年月日并重新组装
    df_inflation["Year_Year"] = df_inflation["Year"].dt.year
    df_inflation["Year_Month"] = df_inflation["Year"].dt.day
    df_inflation["Year_Day"] = df_inflation["Year"].dt.month
    df_inflation["Date"] = pd.to_datetime(
        df_inflation["Year_Year"].astype(str)
        + "-"
        + df_inflation["Year_Month"].astype(str)
        + "-"
        + df_inflation["Year_Day"].astype(str)
    )
    # 删除临时列
    df_inflation.drop(
        columns=["Year", "Year_Year", "Year_Month", "Year_Day"], inplace=True
    )
    # columnd排序，先Date，在CPI
    df_inflation = df_inflation[["Date", "CPI"]]
    transformed["Inflation"] = df_inflation

    # 21. Forward Rate
    df_forward = assumptions_dict["Forward_rates"].copy()
    # 对 Forward Rate 做同样的处理
    df_forward["Year_Year"] = df_forward["Month"].dt.year
    df_forward["Year_Month"] = df_forward["Month"].dt.day
    df_forward["Year_Day"] = df_forward["Month"].dt.month
    df_forward["Month"] = pd.to_datetime(
        df_forward["Year_Year"].astype(str)
        + "-"
        + df_forward["Year_Month"].astype(str)
        + "-"
        + df_forward["Year_Day"].astype(str)
    )
    # 删除临时列
    df_forward.drop(columns=["Year_Year", "Year_Month", "Year_Day"], inplace=True)
    transformed["Forward_rate"] = df_forward

    return transformed


In [ ]:
assumptions = transform_assumptions(assumptions_dict)
print(assumptions["Forward_rate"])



In [ ]:
import modelx as mx

model = mx.read_model("/Users/user/Downloads/Basic_IP_Model_v11_withDateTimeFix&SmokerStatusFix")

model.Assumptions.Forward_rate  

model.Results.cashflow_output_t0()

In [ ]:
# For transformed assumptions DataFrame
print("Transformed Assumptions DataFrame:")
print(assumptions["Forward_rate"].dtypes)
print("\nSample row type:", type(assumptions["Forward_rate"]["Year"].iloc[0]))
print("\n-------------------\n")

# For modelx Assumptions DataFrame
print("ModelX Assumptions DataFrame:")
print(model.Assumptions.Forward_rate.dtypes)
print("\nSample row type:", type(model.Assumptions.Forward_rate["Year"].iloc[0]))

In [ ]:
model.Results.cashflow_output_t0()

In [ ]:
df = model.Results.cashflow_output_t0()
# Check duplicates based on Policy Number
print("Duplicates by Policy Number:")
print(df['Policy Number'].duplicated().any())
print("\nDuplicate Policy Numbers if any:")
print(df[df['Policy Number'].duplicated(keep=False)].sort_values('Policy Number'))

# Check duplicates based on multiple columns
print("\nDuplicates by Policy Number, t, Related_Policy_Group:")
print(df.duplicated(['Policy Number', 't', 'Related_Policy_Group']).any())

# Get a count of each Policy Number
print("\nPolicy Number value counts:")
print(df['Policy Number'].value_counts().sort_index())

In [ ]:
def export_assumptions_to_excel(transformed_dict, output_path):
    """
    Export transformed assumptions dictionary to Excel file with multiple sheets
    
    Args:
        transformed_dict: Dictionary of transformed DataFrames
        output_path: Path where the Excel file should be saved (including filename)
    
    Example:
        export_assumptions_to_excel(transformed_data, "transformed_assumptions.xlsx")
    """
    try:
        # Create Excel writer object
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            # Iterate through the dictionary and write each DataFrame to a sheet
            for sheet_name, df in transformed_dict.items():
                # Ensure the sheet name is valid (Excel has a 31 character limit)
                valid_sheet_name = sheet_name[:31]
                
                # Write the DataFrame to Excel
                df.to_excel(writer, sheet_name=valid_sheet_name, index=False)
                
                # Auto-adjust columns width
                worksheet = writer.sheets[valid_sheet_name]
                for idx, col in enumerate(df.columns):
                    max_length = max(
                        df[col].astype(str).apply(len).max(),  # max length of values
                        len(str(col))  # length of column name
                    )
                    worksheet.column_dimensions[chr(65 + idx)].width = max_length + 2  # Add padding
                
        print(f"Successfully exported to {output_path}")
        print(f"Number of sheets exported: {len(transformed_dict)}")
        
    except Exception as e:
        print(f"Error exporting to Excel: {str(e)}")


export_assumptions_to_excel(
    assumptions, 
    "transformed_assumptions.xlsx"
)


In [72]:
import pandas as pd
import numpy as np

# Step 2: Create a DataFrame with mixed data types
data = {
    'run_number': [1, 1, 2, 2, 3],
    'combined_present_value': np.random.rand(5),  # Random float numbers
    'int_data': [100, 200, 300, 400, 500],  # Integer data
    'string_data': ['a', 'b', 'c', 'd', 'e']  # Non-numeric data
}

df = pd.DataFrame(data)

In [73]:
df

,run_number,combined_present_value,int_data,string_data
0,1,0.438954,100,a
1,1,0.038947,200,b
2,2,0.168038,300,c
3,2,0.150672,400,d
4,3,0.056435,500,e


In [74]:
numeric_sums = df.select_dtypes(include=['float64', 'int64']).sum()


In [91]:
import modelx as mx 



model = mx.read_model("/Users/user/Downloads/Basic_IP_Model_v12 Integration")

model.Results.cashflow_output_t0()


  110.82539784  117.56358203  124.71144781  131.02184707  137.65155254
  144.61672109  151.93432718  159.62220414  167.69908767  176.1846615
  185.09960537  194.46564541  204.30560706  572.63228343  584.0849291
  595.76662768  607.68196023  619.83559944  632.23231142  644.87695765
  657.77449681  670.92998674  684.34858648  698.03555821  711.99626937
  726.23619476  740.76091865  755.57613703  770.68765977  786.10141296
  801.82344122  817.85991005  834.21710825  850.90145041  867.91947942
  885.27786901  902.98342639  921.04309492  939.46395681  958.25323595
  977.41830067  996.96666668 1016.90600002 1037.24412002 1057.98900242
 1079.14878247 1100.73175812   80.16851968   85.86048458   91.95657898
   98.48549609  104.47341425  110.82539784  117.56358203  124.71144781
  131.02184707  137.65155254  144.61672109  151.93432718  159.62220414
  167.69908767  176.1846615   185.09960537  194.46564541  204.30560706
  572.63228343  584.0849291   595.76662768  607.68196023  619.83559944
  632.23

,Policy Number,t,Related_Policy_Group,Related_Policy_Group_Rein,PV_Accident_Claims,PV_Sickness_Claims,PV_Prems,PV_Claims,PV_Expense,PV_Commission,...,I17_Rein_Premiums,I17_Rein_Accident_Claims,I17_Rein_Sickness_Claims,I17_Rein_Claims,I17_Rein_PV_Accident_Claims,I17_Rein_PV_Sickness_Claims,I17_Rein_PV_Prems,I17_Rein_PV_Claims,I17_Rein_BEL,I17_Rein_RA
0,1,0,1,101,180.614869,1517.156600,3733.416842,1697.771468,1482.648533,112.002505,...,0.0,0.0,0.0,0.0,85.754080,705.954857,1103.743817,791.708937,312.034880,-79.170894
210,2,0,2,101,410.882121,4689.254982,20439.607505,5100.137103,2327.667138,618.693634,...,0.0,0.0,0.0,0.0,117.138114,1181.031264,6066.159045,1298.169378,4767.989667,-129.816938
623,3,0,1,101,180.614869,1517.156600,3733.416842,1697.771468,1482.648533,112.002505,...,0.0,0.0,0.0,0.0,85.754080,705.954857,1103.743817,791.708937,312.034880,-79.170894
833,4,0,2,101,410.882121,4689.254982,20439.607505,5100.137103,2327.667138,618.693634,...,0.0,0.0,0.0,0.0,117.138114,1181.031264,6066.159045,1298.169378,4767.989667,-129.816938
1246,5,0,1,101,180.614869,1517.156600,3733.416842,1697.771468,1482.648533,112.002505,...,0.0,0.0,0.0,0.0,85.754080,705.954857,1103.743817,791.708937,312.034880,-79.170894
1456,6,0,2,101,410.882121,4689.254982,20439.607505,5100.137103,2327.667138,618.693634,...,0.0,0.0,0.0,0.0,117.138114,1181.031264,6066.159045,1298.169378,4767.989667,-129.816938
1869,7,0,1,101,180.614869,1517.156600,3733.416842,1697.771468,1482.648533,112.002505,...,0.0,0.0,0.0,0.0,85.754080,705.954857,1103.743817,791.708937,312.034880,-79.170894
2079,8,0,2,101,410.882121,4689.254982,20439.607505,5100.137103,2327.667138,618.693634,...,0.0,0.0,0.0,0.0,117.138114,1181.031264,6066.159045,1298.169378,4767.989667,-129.816938
2492,9,0,1,101,180.614869,1517.156600,3733.416842,1697.771468,1482.648533,112.002505,...,0.0,0.0,0.0,0.0,85.754080,705.954857,1103.743817,791.708937,312.034880,-79.170894
2702,10,0,2,101,410.882121,4689.254982,20439.607505,5100.137103,2327.667138,618.693634,...,0.0,0.0,0.0,0.0,117.138114,1181.031264,6066.159045,1298.169378,4767.989667,-129.816938


In [99]:
pv=model.Results.cashflow_output()
sums = pv.select_dtypes(include=['float64', 'int64']).sum()

print(sums["I17_PV_Prems"])

7446.8996983966335


In [101]:
rpg = model.Results.rpg_aggregate()

# stacked_results_rpg = rpg.groupby(['variable'])['value'].sum().reset_index()
# print(stacked_results_rpg)
rpg

,RPG,variable,value
0,1,PV_Prems,"2,168,037.44"
1,1,I17_BEL,(301.00)
2,1,I17_PV_Commission,24.83
3,1,I17_PV_Expense,460.35
4,1,I17_PV_Claims,41.58
5,1,I17_PV_Prems,827.77
6,1,RA,"123,814.59"
7,1,I17_RA,4.16
8,1,PV_Commission,"65,041.12"
9,1,PV_Expense,"814,431.61"


In [102]:
6619.13 + 827.77

7446.9